 ### **Question A | Non-parametric VaR estimation**

>a – From the time series of the daily prices of the stock Natixis between January 2015 and December
2016, provided with TD1, estimate a historical VaR on price returns at a one-day horizon for a given
probability level (this probability is a parameter which must be changed easily). You must base your
VaR on a non-parametric distribution (biweight Kernel, that is 𝐾 is the derivative of the logistic function$$K(x) = \frac{15}{16} (1 - x^2)^2 \mathbb{1}_{|x| \le 1}$$

In [92]:
import pandas as pd
import numpy as np

In [93]:
#We load the data 
df = pd.read_csv("Natixis Stock.csv", header=None, sep=r"\s+", names=["Date", "Price"])
df.head()

,Date,Price
0,02/01/2015,"5,621"
1,05/01/2015,"5,424"
2,06/01/2015,"5,329"
3,07/01/2015,"5,224"
4,08/01/2015,"5,453"


In [94]:
#Parsing dates correctly and handling the decimal format
df["Date"]=pd.to_datetime(df["Date"], format="%d/%m/%Y", errors="coerce")
df["Price"]=df["Price"].str.replace(",",".").astype(float)

In [117]:
#Some informations about the dataset
print(df.head(5))
print(df.tail(5))
print(df.describe())
#Verification of data
print("Number of NaN price:", df["Price"].isna().sum())
print("Number of NaN price:", df["Date"].isna().sum())

        Date  Price    return
1 2015-01-05  5.424 -0.035047
2 2015-01-06  5.329 -0.017515
3 2015-01-07  5.224 -0.019704
4 2015-01-08  5.453  0.043836
5 2015-01-09  5.340 -0.020723
           Date  Price    return
1018 2018-12-21  4.045 -0.001481
1019 2018-12-24  4.010 -0.008653
1020 2018-12-27  3.938 -0.017955
1021 2018-12-28  4.088  0.038090
1022 2018-12-31  4.119  0.007583
                                Date        Price       return
count                           1022  1022.000000  1022.000000
mean   2016-12-31 04:00:56.360078336     5.684662    -0.000097
min              2015-01-05 00:00:00     3.077000    -0.171325
25%              2016-01-04 06:00:00     4.925500    -0.010602
50%              2016-12-29 12:00:00     5.784000    -0.000506
75%              2017-12-28 18:00:00     6.532500     0.011274
max              2018-12-31 00:00:00     7.744000     0.090281
std                              NaN     1.021532     0.020286
Number of NaN price: 0
Number of NaN price: 0


We use arithmetic return because it is the most accurate indicator for measuring the actual loss in value of a single asset over a short time horizon (1 day).

$$R_t = \frac{P_t - P_{t-1}}{P_{t-1}}$$

In [96]:
df["return"]=df["Price"].pct_change() 
df = df.iloc[1:] #Del the Nan values of the first row
df

,Date,Price,return
1,2015-01-05,5.424,-0.035047
2,2015-01-06,5.329,-0.017515
3,2015-01-07,5.224,-0.019704
4,2015-01-08,5.453,0.043836
5,2015-01-09,5.340,-0.020723
...,...,...,...
1018,2018-12-21,4.045,-0.001481
1019,2018-12-24,4.010,-0.008653
1020,2018-12-27,3.938,-0.017955
1021,2018-12-28,4.088,0.038090


In [97]:
# We select the daily prices of the stock Natixis between January 2015 and December 2016 for question a
df_a =df[(df["Date"]>="2015-01-01") & (df["Date"]<="2016-12-31")] 
print(df_a.tail(5))

          Date  Price    return
508 2016-12-23  5.376 -0.008118
509 2016-12-27  5.380  0.000744
510 2016-12-28  5.379 -0.000186
511 2016-12-29  5.328 -0.009481
512 2016-12-30  5.360  0.006006


In [98]:
#Probability of level alpha (1-alpha*100) 
alpha=0.05 #For alpha=0.05, the probability level is 1-0.05*100=95%

To calculate the VaR, we need the estimated cumulative distribution function $\hat{F}(x)$. According to the course:


$$\hat{F}(x) = \frac{1}{n} \sum_{i=1}^{n} \mathcal{K} \left( \frac{x - X_i}{h} \right)$$


Where $$\mathcal{K}(u) = \int_{-\infty}^{u} K(t) dt$$ is the kernel integral. For the Biweight kernel:


* If $u < -1$: $\mathcal{K}(u) = 0$
* If $u > 1$: $\mathcal{K}(u) = 1$
* If $u \in [-1, 1]$:

And $$K(t) = \frac{15}{16}(1 - t^2)^2 \mathbb{1}_{|t| \leq 1}$$



Thus $$\mathcal{K}(u) = \int_{-1}^{u} \frac{15}{16}(1 - 2t^2 + t^4) dt = \frac{1}{2} + \frac{15}{16} \left( u - \frac{2}{3}u^3 + \frac{1}{5}u^5 \right)$$


The VaR at level $\alpha$ is then the value $x$ such that $\hat{F}(x) = \alpha$.

In [99]:
#Calculation of the Biweight kernel integral from -inf to u
def K(u):
    if u<-1:
        return 0
    if u>1:
        return 1
    return 0.5+(15/16)*(u-(2/3)*(u**3)+(1/5)*(u**5))

The choice of the bandwidth parameter $h$ is crucial as it determines the bias-variance tradeoff of our estimator:

* A bandwidth that is too small leads to a distribution that is too close to the historical data.
* A bandwidth that is too large smooths out the specific features of the distribution.

In this project, we could use Silverman’s rule of thumb as a starting point, but it was originally calibrated for a Gaussian kernel:

$$h_{Silverman} = 1.06 \cdot \hat{\sigma} \cdot n^{-1/5}$$

Thus, to remain consistent with the course, we will use **$h = 0.003$**.

In [100]:
h = 0.003
#Estimation of the CDF
def F(x,rend):
    u=(x-rend)/h
    somme=sum(K(val) for val in u)
    return somme/len(rend)

In [101]:
#We use a dichotomy algorithm to calculate VaR such that F(VaR) = alpha
def dichotomie(rend,alpha):
    a=-1
    b=1
    eps=1e-9
    while abs(b - a)>eps:
        m = (a + b)/2
        if F(m,rend) == 1-alpha:
            return m
        elif F(m,rend) < 1-alpha:
            a = m
        else:
            b = m
    return (a + b)/ 2

Var_Nonpara= dichotomie(df_a["return"], alpha)
print(f"VaR Non-parametric at {100-alpha*100}% = {Var_Nonpara:.2%}")

VaR Non-parametric at 95.0% = 3.86%


So we are sure at 95% (1-alpha*100%) to not lose more than 3.86% at one-day horizon. 
As we use a biweight kernel with a bandwidth of h=0.003 (course material) on the 2015-2016 period, it provides a smoothed estimation of the lower tail of the return distribution. 
In contrast to the empirical quantile method, this non-parametric approach leverages the local density of observations.

>b – Which proportion of price returns between January 2017 and December 2018 does exceed the VaR
threshold defined in the previous question? Do you validate the choice of this non-parametric VaR?

We first calculate the returns for the test period. Then, we count how many of these returns $R_t$ satisfy the condition:

$$R_t < -VaR_{1-\alpha}$$

where $VaR_{1-\alpha}$ is the positive value calculated in part (a).
The proportion of exceptions $\hat{p}$ is given by:

$$\hat{p} = \frac{1}{N_{test}} \sum_{t=1}^{N_{test}} \mathbb{1}_{\{R_t < -VaR_{1-\alpha}\}}$$

In [110]:
#Test dataset to validate or not the choice of the VaR
df_b = df[(df["Date"] >= "2017-01-01") & (df["Date"] <= "2018-12-31")]
#We use -Var_Nonpara because Var_Nonpara is the positive loss
exceeding_returns=df_b[df_b["return"]< -Var_Nonpara]
proportion = len(exceeding_returns)/len(df_b)
print(f"The proportion of returns exceeding VaR is : {proportion:.2%}")

The proportion of returns exceeding VaR is : 1.57%


We observe that 1.57%<5%. This means that in the 2017-2018 period, the losses exceeded the 2015-2016 risk estimate significantly less often than predicted.
Thus, the model is conservative because as we can see, it overestimates the actual risk encountered in 2017-2018.

To conclude, we partially validate the choice. Indeed, the model is safe (no underestimation of risk), but it lacks efficiency as it provides a threshold that is too high for the actual market conditions. 

We suggest to use a weighted historical VaR or an adaptive bandwidth as seen in the course to give more importance to recent observations and better adapt to the change of regimes in the market.

 ### **Question B | Expected shortfall**

>Calculate the expected shortfall for the VaR calculated in question A. How is the result, compared to the VaR?

To be consistent with the previous questions, we will not calculate the ES using a simple empirical average, but using the same probability density as before.
The kernel smooths out the effects of isolated extreme values to obtain a more robust measurement.

We defined the Expected Shortfall (ES) as:

$$ES_{\alpha} = \frac{1}{\alpha} \int_{-\infty}^{q_{\alpha}} x \hat{f}(x) dx$$

Using our biweight kernel estimator, we can simplifies this integral to a weighted sum:

$$ES_{returns} = \frac{1}{n \alpha} \sum_{i=1}^{n} \left[ X_i \mathcal{K}(u_i) + h J(u_i) \right]$$

Where:

* $u_i = \frac{q_{\alpha} - X_i}{h}$
* $\mathcal{K}(u)$ is the kernel integral (already implemented for the VaR calculation)
* $J(u) = \int_{-1}^{u} t K(t) dt$ is the first moment of the kernel

For the biweight kernel, the moment primitive is:

$$J(u) = \frac{15}{16} \left( \frac{1}{2}u^2 - \frac{1}{2}u^4 + \frac{1}{6}u^6 - \frac{1}{6} \right)$$

In [ ]:
#Calculation of the moment primitive for the biweight kernel
def J(u):
    if u<-1 or u>1:  # The integral over the entire support of a symmetric kernel = 0
        return 0
    return (15/16)*(0.5*u**2-0.5*u**4+(1/6)*u**6-1/6)

In [ ]:
# Expected Shortfall calculation
def ES(rend):  #rend is the array of historical returns 
    n=len(rend)
    #Var_Nonpara has to be the negative quantile
    u=(-Var_Nonpara-rend)/h  #u is the standardized distance between the qunatile and each historical observation
    XK=rend*np.array([K(i) for i in u]) #return weighted by the cumulative probability of the kernel
    hJ=h*np.array([J(i) for i in u])
    es_return=np.mean(XK + hJ)/alpha # ES formula for a kernel estimator(explained above)
    return -es_return    # The loss has to be >0

ES_Nonpara = ES(df_a["return"])
print(f"Expected Shortfall: {ES_Nonpara:.2%}")

Expected Shortfall: 5.23%


The Expected Shortfall (5.23%) is higher than the VaR (3.86%). This result was expected since the ES measures the average of all losses in the tail of the distribution, and not just the cut-off point.

The ES/VaR ratio here is approximately 1.41. This indicates the danger of tail risk. Indeed, if returns were perfectly normal, this ratio would be lower. The biweight kernel therefore better captures the risk of extreme losses specific to the stock, which VaR alone does not take into account.